In [ ]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen

In [ ]:
url_base = 'https://www.beermile.com/display/'
max_event_id = 151381

In [ ]:
%%time

# Find which events are actually races
is_event = []
not_event = []


# Look for all events by numerical ID
for i in range(1, max_event_id):
    # Website uses sequential 
    event = f'event_{i}'
    url = f'{url_base}{event}'
    result = requests.get(url)
    page = urlopen(result.url).read()
    a = bs(page, 'html.parser')
    
    
    if (a.title.string == 'Race Not Available'):
        not_event.append(i)
        with open('not_events.txt', 'w') as g:
            g.write(f'{item}\n')
    else:
        is_event.append(i)
        with open('events.txt', 'w') as f:
            f.write(f'{item}\n')
            
    print(i)
